# Выпускной проект по SQL

 ## Цели исследования
 
Проанализировать базу данных сервиса для чтения книг по подписке. В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

## Описание данных

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

![jupyter](https://concrete-web-bad.notion.site/image/https%3A%2F%2Fs3-us-west-2.amazonaws.com%2Fsecure.notion-static.com%2F069818d1-0e5c-4d87-a461-0de584ab9c33%2FUntitled_(33).png?id=bd53c8db-b4fd-49eb-8cc9-572ebb3c9163&table=block&spaceId=9e4bd47b-c6e6-4ca3-bcee-279794b47315&width=2000&userId=&cache=v2)

Импорт библиотек

In [1]:
import pandas as pd
from sqlalchemy import create_engine

Подключение к базе данных

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

## Исследование таблиц

Создадим функцию для выполнения запроса:

In [3]:
def get_query(query):
    result = pd.io.sql.read_sql(query, con = engine)
    return result

Выведем первые строки каждой из таблиц

In [4]:
query = '''SELECT * 
        FROM books 
        LIMIT 1'''
get_query(query)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93


In [5]:
query = '''SELECT * 
        FROM authors 
        LIMIT 1'''
get_query(query)

,author_id,author
0,1,A.S. Byatt


In [6]:
query = '''SELECT * 
        FROM publishers 
        LIMIT 1'''
get_query(query)

,publisher_id,publisher
0,1,Ace


In [7]:
query = '''SELECT * 
        FROM ratings 
        LIMIT 1'''
get_query(query)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4


In [8]:
query = '''SELECT * 
        FROM reviews 
        LIMIT 1'''
get_query(query)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...


## Задача 1
#### Посчитайте, сколько книг вышло после 1 января 2000 года

In [9]:
query = '''SELECT 
        COUNT (*) 
        FROM books 
        WHERE publication_date > TO_DATE('2000-01-01', 'YYYY-MM-DD');
        '''
get_query(query)

,count
0,819


#### Вывод

В таблице books содержится информация о 819 книгах вышедших после 1 января 2000 года.

## Задача 2
#### Для каждой книги посчитайте количество обзоров и среднюю оценку

In [10]:
query = '''SELECT b.book_id, b.title, COUNT(DISTINCT rev.review_id), ROUND(AVG(rat.rating), 2)
           FROM books b
           LEFT JOIN reviews rev ON b.book_id = rev.book_id
           LEFT JOIN ratings rat ON b.book_id = rat.book_id
           GROUP BY b.book_id, b.title
           ORDER BY COUNT(rev.review_id) DESC;
        '''
get_query(query)

,book_id,title,count,round
0,948,Twilight (Twilight #1),7,3.66
1,750,The Hobbit or There and Back Again,6,4.13
2,673,The Catcher in the Rye,6,3.83
3,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.41
4,299,Harry Potter and the Chamber of Secrets (Harry...,6,4.29
...,...,...,...,...
995,221,Essential Tales and Poems,0,4.00
996,387,Leonardo's Notebooks,0,4.00
997,672,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.00
998,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.67


#### Вывод

Лидером по количеству обзоров (7) стала первая часть Сумерек. При этом рейтинг у неё довольно средний - 3.66. Есть книги без обзоров.

## Задача 3
#### Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры

In [11]:
query = '''SELECT p.publisher,
           COUNT(xxx.book_id) AS books_cnt
           FROM publishers p
           JOIN (SELECT *
                      FROM books b
                      WHERE b.num_pages > 50) AS xxx
                      ON xxx.publisher_id = p.publisher_id
           GROUP BY p.publisher
           ORDER BY COUNT(xxx.book_id) DESC
           LIMIT 1;
           '''
get_query(query)

,publisher,books_cnt
0,Penguin Books,42


#### Вывод
Больше всего книг выпустило издательство Penguin Books	- 42.

## Задача 4
#### Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

In [12]:
query = '''SELECT
           xxx.author AS author,
           ROUND(AVG(xxx.avg_rating), 2) AS avg_rating
           FROM
           (SELECT a.author AS author, b.book_id AS id, AVG(rat.rating) AS avg_rating,
           COUNT(rat.rating) AS cnt_rating
           FROM authors a
           JOIN books b ON b.author_id = a.author_id
           JOIN ratings rat ON rat.book_id = b.book_id
           GROUP BY author, id
           HAVING COUNT(rat.rating) >= 50) AS xxx
           GROUP BY author
           ORDER BY avg_rating DESC
           LIMIT 1;
        '''
get_query(query)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.28


#### Вывод
Самая высокая средняя оценка книг у автора J.K. Rowling/Mary GrandPré

## Задача 5
#### Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [13]:
query = '''SELECT AVG(count)
           FROM
          (SELECT COUNT(text)
           FROM reviews rev
           JOIN (SELECT username, COUNT(rating_id) as cnt
           FROM ratings rat
           GROUP BY username) AS cnt_rating
           ON cnt_rating.username = rev.username
           WHERE cnt > 50
           GROUP BY rev.username) as count; 
        '''
get_query(query)

,avg
0,24.333333


#### Вывод

Среднее количество о пользователей, поставивших больше 50 оценок - 24.